In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import operator

In [3]:
def load_dataset(month="2020年6月", date=20200613, grouped=True):
    root_path = "/Users/liyurun/Downloads/OneDrive_1_11-18-2020/提供データ/配達実績/" + month + ".xls"
    original = pd.read_excel(root_path)

    # 指定した日付のみを抽出
    data = original[original["完了日付"] == date]
    data = data.drop(["完了日付"], axis=1)

    # 緯度・経度情報を統合
    data_0 = data[data["緯度＿完了優先"] == 0]
    data_0["緯度"] = data_0["緯度＿住所より"]
    data_0["経度"] = data_0["経度＿住所より"]

    data_1 = data[data["緯度＿完了優先"] != 0]
    data_1["緯度"] = data_1["緯度＿完了優先"]
    data_1["経度"] = data_1["経度＿完了優先"]

    data = pd.concat([data_0, data_1], axis=0)

    # コード0を削除
    data = data[data["配達乗務員\nコード"] != 0]

    # 必要なcolumnのみ抽出
    col = ["配達乗務員\nコード", "完了時刻", "緯度", "経度", "個数", "重量"]
    data = data[col]

    # ソート
    data = data.sort_values(["配達乗務員\nコード", "完了時刻"])
    data.reset_index(drop=True, inplace=True)

    # 緯度・経度に関して重複する部分をグループ化する場合
    if grouped:
        _data = pd.DataFrame()
        for driver_id in get_id_list(data):
            data_per_driver = data[data["配達乗務員\nコード"] == driver_id]
            data_per_driver = grouped_duplicate(data_per_driver)
            _data = _data.append(data_per_driver, ignore_index=True)
        data = _data
        data.reset_index(drop=True, inplace=True)

    return data


# 緯度・経度に関して重複する部分をグループ化
def grouped_duplicate(data):
    # 緯度・経度が同じものに関してグループ化
    df_1 = data.groupby(["緯度", "経度"]).max()  # df_1:グループの中の最大値を取得
    df_2 = data.groupby(["緯度", "経度"]).min()  # df_2:グループの中の最小値を取得
    df_3 = data.groupby(["緯度", "経度"]).sum()  # df_3:グループの中の総和を取得

    df_1 = df_1[["配達乗務員\nコード", "完了時刻"]]
    df_2 = df_2[["完了時刻"]].rename(columns={"完了時刻": "1st完了時刻"})
    df_3 = df_3[["個数", "重量"]]

    # 結合
    df = pd.concat([df_1, df_2, df_3], axis=1)

    # 緯度・経度がindexに入ってるためcolumnへ
    df["緯度"] = [df.index[i][0] for i in range(len(df.index))]
    df["経度"] = [df.index[i][1] for i in range(len(df.index))]

    # 順序を入れ替える
    df = df[["配達乗務員\nコード", "1st完了時刻", "完了時刻", "緯度", "経度", "個数", "重量"]]
    df = df.sort_values("完了時刻")
    df.reset_index(drop=True, inplace=True)

    return df


# dataに含まれるドライバーのIDを取得
def get_id_list(data):
    data_grouped = data.groupby("配達乗務員\nコード")
    id_list = list(data_grouped.size().index)
    return id_list


# データから指定したIDのドライバー1人のデータを取得
def get_data_per_driver(data, driver_id):
    data_per_driver = data[data["配達乗務員\nコード"] == driver_id]
    data_per_driver.reset_index(drop=True, inplace=True)
    return data_per_driver